In [1]:
# Bag of apps categories
# Bag of labels categories
# Include phone brand and model device

print("Initialize libraries")

import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.metrics import log_loss
from sklearn.cluster import DBSCAN
from sklearn import metrics as skmetrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from collections import Counter
from keras.layers.advanced_activations import PReLU
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import ensemble
from sklearn.decomposition import PCA
import os
import gc
from scipy import sparse
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, SelectKBest
from sklearn import ensemble
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

Initialize libraries


Using Theano backend.


In [2]:
#------------------------------------------------- Write functions ----------------------------------------

def rstr(df): return df.dtypes, df.head(3) ,df.apply(lambda x: [x.unique()]), df.apply(lambda x: [len(x.unique())]),df.shape

def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

In [3]:

#------------------------------------------------ Read data from source files ------------------------------------

seed = 7
np.random.seed(seed)
datadir = ''

print("### ----- PART 1 ----- ###")

# Data - Events data
# Bag of apps
print("# Read app events")
app_events = pd.read_csv(os.path.join(datadir,'app_events.csv'), dtype={'device_id' : np.str})
app_events.head(5)
app_events.info()
#print(rstr(app_events))

# remove duplicates(app_id)
app_events= app_events.groupby("event_id")["app_id"].apply(
    lambda x: " ".join(set("app_id:" + str(s) for s in x)))
app_events.head(5)

print("# Read Events")
events = pd.read_csv(os.path.join(datadir,'events.csv'), dtype={'device_id': np.str})
events.head(5)
events["app_id"] = events["event_id"].map(app_events)
events = events.dropna()
del app_events

events = events[["device_id", "app_id"]]
events.info()
# 1Gb reduced to 34 Mb

# remove duplicates(app_id)
events.loc[:,"device_id"].value_counts(ascending=True)

events = events.groupby("device_id")["app_id"].apply(
    lambda x: " ".join(set(str(" ".join(str(s) for s in x)).split(" "))))
events = events.reset_index(name="app_id")

# expand to multiple rows
events = pd.concat([pd.Series(row['device_id'], row['app_id'].split(' '))
                    for _, row in events.iterrows()]).reset_index()
events.columns = ['app_id', 'device_id']
events.head(5)
f3 = events[["device_id", "app_id"]]    # app_id

print("#Part1 formed")

### ----- PART 1 ----- ###
# Read app events
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32473067 entries, 0 to 32473066
Data columns (total 4 columns):
event_id        int64
app_id          int64
is_installed    int64
is_active       int64
dtypes: int64(4)
memory usage: 991.0 MB
# Read Events
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1488096 entries, 1 to 3252947
Data columns (total 2 columns):
device_id    1488096 non-null object
app_id       1488096 non-null object
dtypes: object(2)
memory usage: 34.1+ MB
#Part1 formed


In [4]:
##################
#   App labels
##################


print("### ----- PART 2 ----- ###")

print("# Read App labels")
app_labels = pd.read_csv(os.path.join(datadir,'app_labels.csv'))
label_cat = pd.read_csv(os.path.join(datadir,'label_categories.csv'))
app_labels.info()
label_cat.info()
label_cat=label_cat[['label_id','category']]

app_labels=app_labels.merge(label_cat,on='label_id',how='left')
app_labels.head(3)
events.head(3)
#app_labels = app_labels.loc[app_labels.smaller_cat != "unknown_unknown"]

#app_labels = app_labels.groupby("app_id")["category"].apply(
#    lambda x: ";".join(set("app_cat:" + str(s) for s in x)))
app_labels = app_labels.groupby(["app_id","category"]).agg('size').reset_index()
app_labels = app_labels[['app_id','category']]
print("# App labels done")


# Remove "app_id:" from column
print("## Handling events data for merging with app lables")
events['app_id'] = events['app_id'].map(lambda x : x.lstrip('app_id:'))
events['app_id'] = events['app_id'].astype(str)
app_labels['app_id'] = app_labels['app_id'].astype(str)
app_labels.info()

print("## Merge")

events= pd.merge(events, app_labels, on = 'app_id',how='left').astype(str)
#events['smaller_cat'].unique()

# expand to multiple rows
print("#Expand to multiple rows")
#events= pd.concat([pd.Series(row['device_id'], row['category'].split(';'))
#                    for _, row in events.iterrows()]).reset_index()
#events.columns = ['app_cat', 'device_id']
#events.head(5)
#print(events.info())

events= events.groupby(["device_id","category"]).agg('size').reset_index()
events= events[['device_id','category']]
events.head(10)
print("# App labels done")

f5 = events[["device_id", "category"]]    # app_id
# Can % total share be included as well?
print("# App category part formed")

### ----- PART 2 ----- ###
# Read App labels
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459943 entries, 0 to 459942
Data columns (total 2 columns):
app_id      459943 non-null int64
label_id    459943 non-null int64
dtypes: int64(2)
memory usage: 7.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 2 columns):
label_id    930 non-null int64
category    927 non-null object
dtypes: int64(1), object(1)
memory usage: 14.6+ KB
# App labels done
## Handling events data for merging with app lables
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457777 entries, 0 to 457776
Data columns (total 2 columns):
app_id      457777 non-null object
category    457777 non-null object
dtypes: object(2)
memory usage: 7.0+ MB
## Merge
#Expand to multiple rows
# App labels done
# App category part formed


In [5]:
##################
#   Phone Brand
##################
print("### ----- PART 3 ----- ###")

print("# Read Phone Brand")
pbd = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'),
                  dtype={'device_id': np.str})
pbd.drop_duplicates('device_id', keep='first', inplace=True)

##################
#  Train and Test
##################
print("# Generate Train and Test")

train = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),
                    dtype={'device_id': np.str})
train.drop(["age", "gender"], axis=1, inplace=True)

test = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),
                   dtype={'device_id': np.str})
test["group"] = np.nan


split_len = len(train)

# Group Labels
Y = train["group"]
lable_group = LabelEncoder()
Y = lable_group.fit_transform(Y)
device_id = test["device_id"]

# Concat
Df = pd.concat((train, test), axis=0, ignore_index=True)

print("### ----- PART 4 ----- ###")

Df = pd.merge(Df, pbd, how="left", on="device_id")
Df["phone_brand"] = Df["phone_brand"].apply(lambda x: "phone_brand:" + str(x))
Df["device_model"] = Df["device_model"].apply(
    lambda x: "device_model:" + str(x))

### ----- PART 3 ----- ###
# Read Phone Brand
# Generate Train and Test
### ----- PART 4 ----- ###


In [6]:
###################
#  Concat Feature
###################

print("# Concat all features")

f1 = Df[["device_id", "phone_brand"]]   # phone_brand
f2 = Df[["device_id", "device_model"]]  # device_model

events = None
Df = None

f1.columns.values[1] = "feature"
f2.columns.values[1] = "feature"
f5.columns.values[1] = "feature"
f3.columns.values[1] = "feature"

FLS = pd.concat((f1, f2, f3, f5), axis=0, ignore_index=True)

FLS.info()

###################
# User-Item Feature
###################
print("# User-Item-Feature")

device_ids = FLS["device_id"].unique()
feature_cs = FLS["feature"].unique()

data = np.ones(len(FLS))
len(data)

dec = LabelEncoder().fit(FLS["device_id"])
row = dec.transform(FLS["device_id"])
col = LabelEncoder().fit_transform(FLS["feature"])
sparse_matrix = sparse.csr_matrix(
    (data, (row, col)), shape=(len(device_ids), len(feature_cs)))
sparse_matrix.shape
sys.getsizeof(sparse_matrix)

sparse_matrix = sparse_matrix[:, sparse_matrix.getnnz(0) > 0]
print("# Sparse matrix done")

del FLS
del data
f1 = [1]
f5 = [1]
f2 = [1]
f3 = [1]

events = [1]

# Concat all features
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6677856 entries, 0 to 6677855
Data columns (total 2 columns):
device_id    object
feature      object
dtypes: object(2)
memory usage: 101.9+ MB
# User-Item-Feature
# Sparse matrix done


In [7]:
##################
#      Data
##################

print("# Split data")
train_row = dec.transform(train["device_id"])
train_sp = sparse_matrix[train_row, :]

test_row = dec.transform(test["device_id"])
test_sp = sparse_matrix[test_row, :]

X_train, X_val, y_train, y_val = train_test_split(
    train_sp, Y, train_size=0.999, random_state=10)

##################
#   Feature Sel
##################
print("# Feature Selection")
#selector = SelectPercentile(f_classif, percentile=53)

#selector.fit(X_train, y_train)
#X_train.shape
#X_train = selector.transform(X_train)
#X_train.shape
#X_val = selector.transform(X_val)
#X_val.shape

# Selection using chi-square
# selector = SelectKBest(chi2, k=11155).fit(X_train, y_train)
# X_train.shape
# X_train = selector.transform(X_train)
# X_train.shape
# X_val = selector.transform(X_val)
# X_val.shape

print("# Num of Features: ", X_train.shape[1])

# Split data
# Feature Selection
('# Num of Features: ', 21425)


In [8]:
import xgboost as xgb
import time

def run_xgb(train, test, target, eta=0.5, random_state=0):
    #eta = 0.1
    max_depth = 5
    subsample = 0.7
    colsample_bytree = 0.7
    start_time = time.time()

    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth, subsample, colsample_bytree))
    params = {
        "objective": "multi:softprob",
        "num_class": 12,
        "booster" : "gbtree",
        "eval_metric": "mlogloss",
        "eta": eta,
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": 1,
        "seed": random_state,
    }
    num_boost_round = 2000
    early_stopping_rounds = 100
    test_size = 0.3

    X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=test_size, random_state=random_state)
    # TODO change split 
    print('Length train:', X_train.shape[0])
    print('Length valid:', X_valid.shape[0])
    #y_train = X_train[target]
    #y_valid = X_valid[target]
    # TODO delete upper code
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    # sparse matrix ???

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

    print("Validating...")
    check = gbm.predict(xgb.DMatrix(X_valid), ntree_limit=gbm.best_iteration)
    score = log_loss(y_valid.tolist(), check)

    print("Predict test set...")
    test_prediction = gbm.predict(xgb.DMatrix(test), ntree_limit=gbm.best_iteration)

    print('Training time: {} minutes'.format(round((time.time() - start_time)/60, 2)))
    return test_prediction.tolist(), score

In [9]:
last = pd.read_csv('keras_2.26110777855.csv')
last['device_id'] = last.device_id.astype(str)
last = last.set_index("device_id")

In [29]:
def fifth_model():
    # create model
    model = Sequential()
    model.add(Dense(152, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.42))
    model.add(Dense(52, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.18))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_5 = fifth_model()

fit_5 = model_5.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_5 = model_5.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_5)))

Epoch 1/18


/home/ubuntu/anaconda/lib/python2.7/site-packages/keras/engine/training.py:1463: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


17s - loss: 2.4295 - acc: 0.1338 - val_loss: 2.4061 - val_acc: 0.0667
Epoch 2/18
17s - loss: 2.3543 - acc: 0.1638 - val_loss: 2.3685 - val_acc: 0.0667
Epoch 3/18
17s - loss: 2.3158 - acc: 0.1770 - val_loss: 2.3241 - val_acc: 0.1067
Epoch 4/18
17s - loss: 2.2987 - acc: 0.1824 - val_loss: 2.3146 - val_acc: 0.1067
Epoch 5/18
17s - loss: 2.2817 - acc: 0.1881 - val_loss: 2.2957 - val_acc: 0.0667
Epoch 6/18
17s - loss: 2.2738 - acc: 0.1926 - val_loss: 2.3014 - val_acc: 0.0667
Epoch 7/18
17s - loss: 2.2638 - acc: 0.1965 - val_loss: 2.2922 - val_acc: 0.0933
Epoch 8/18
17s - loss: 2.2590 - acc: 0.2012 - val_loss: 2.2901 - val_acc: 0.1067
Epoch 9/18
17s - loss: 2.2503 - acc: 0.2052 - val_loss: 2.2771 - val_acc: 0.1200
Epoch 10/18
17s - loss: 2.2382 - acc: 0.2064 - val_loss: 2.2794 - val_acc: 0.0800
Epoch 11/18
17s - loss: 2.2383 - acc: 0.2075 - val_loss: 2.2852 - val_acc: 0.1333
Epoch 12/18
17s - loss: 2.2294 - acc: 0.2084 - val_loss: 2.2758 - val_acc: 0.1333
Epoch 13/18
17s - loss: 2.2256 - acc

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [31]:
print("# Final prediction")
scores_55 = model_5.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_55 = pd.DataFrame(scores_55 , columns=lable_group.classes_)
result_55["device_id"] = device_id
print(result_55.head(1))
result_55 = result_55.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

# Final prediction
       F23-    F24-26    F27-28    F29-32    F33-42      F43+      M22-  \
0  0.000035  0.000198  0.000707  0.003975  0.021523  0.033455  0.000436   

     M23-26    M27-28    M29-31    M32-38      M39+            device_id  
0  0.011628  0.028034  0.105885  0.270119  0.524005  1002079943728939269  


In [ ]:
# дальше разбавляем сетками, 10% пятой дало небольшой рост
# xgboost не дает прироста
# нужно не меньше 5-10 позиция для серебра

In [35]:
tot = 0.9*last + 0.1*result_55

In [36]:
tot[tot<0.001]=0.001 # 
tot[tot>0.999]=0.999 # 

In [37]:
tot.to_csv('keras_test' + '.csv', index=True, index_label='device_id')

In [ ]:
res, score = run_xgb(train_sp, test_sp, Y, eta=0.07, random_state=0)
print score

In [ ]:
res1, score1 = run_xgb(train_sp, test_sp, Y, eta=0.07, random_state=100)
print score1

In [ ]:
res2, score2 = run_xgb(train_sp, test_sp, Y, eta=0.07, random_state=142)
print score2

In [ ]:
result_0 = pd.DataFrame(res, columns=lable_group.classes_)
result_0["device_id"] = device_id
print(result_0.head(1))
result_0 = result_0.set_index("device_id")

In [ ]:
result_01 = pd.DataFrame(res1, columns=lable_group.classes_)
result_01["device_id"] = device_id
print(result_01.head(1))
result_01 = result_01.set_index("device_id")

In [ ]:
result_02 = pd.DataFrame(res2, columns=lable_group.classes_)
result_02["device_id"] = device_id
print(result_02.head(1))
result_02 = result_02.set_index("device_id")

In [ ]:
#result.to_csv('xgb.csv', index=True, index_label='device_id')

In [ ]:
##################
#  Build Model
##################


#act = keras.layers.advanced_activations.PReLU(init='zero', weights=None)

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(150, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model


model = baseline_model()

fit= model.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69984,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )


# evaluate the model
scores_val = model.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val)))


#print("# Final prediction")
#scores = model.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
#result = pd.DataFrame(scores , columns=lable_group.classes_)
#result["device_id"] = device_id
#print(result.head(1))
#result = result.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result.to_csv('keras_' + str(log_loss(y_val, scores_val)) + '.csv', index=True, index_label='device_id')


print("Done")

In [ ]:
def second_model():
    # create model
    model = Sequential()
    model.add(Dense(155, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_2 = second_model()

fit_2 = model_2.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69784,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

scores_val_2 = model_2.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])

In [ ]:
def third_model():
    # create model
    model = Sequential()
    model.add(Dense(145, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_3 = third_model()

fit_3 = model_3.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

scores_val_3 = model_3.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_3)))

In [ ]:
def forth_model():
    # create model
    model = Sequential()
    model.add(Dense(147, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_4 = forth_model()

fit_4 = model_4.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69984,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_4 = model_4.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_4)))

In [ ]:
def fifth_model():
    # create model
    model = Sequential()
    model.add(Dense(152, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.42))
    model.add(Dense(52, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.18))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_5 = fifth_model()

fit_5 = model_5.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_5 = model_5.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_5)))

In [ ]:
def sixth_model():
    # create model
    model = Sequential()
    model.add(Dense(153, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.42))
    model.add(Dense(51, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_6 = fifth_model()

fit_6 = model_6.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_6 = model_6.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_5)))

In [ ]:
def seventh_model():
    # create model
    model = Sequential()
    model.add(Dense(144, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_7 = seventh_model()

fit_7 = model_7.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_7 = model_7.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_7)))

In [ ]:
def eight_model():
    # create model
    model = Sequential()
    model.add(Dense(145, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(51, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_8 = eight_model()

fit_8 = model_8.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69784,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_8 = model_8.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_8)))

In [ ]:
'''def ninth_model():
    # create model
    model = Sequential()
    model.add(Dense(156, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(51, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_9 = ninth_model()

fit_9 = model_9.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_9 = model_9.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_9)))'''

In [ ]:
print("# Final prediction")
scores_3 = model_3.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_3 = pd.DataFrame(scores_3 , columns=lable_group.classes_)
result_3["device_id"] = device_id
print(result_3.head(1))
result_3 = result_3.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

In [ ]:
print("# Final prediction")
scores_2 = model_2.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_2 = pd.DataFrame(scores_2 , columns=lable_group.classes_)
result_2["device_id"] = device_id
print(result_2.head(1))
result_2 = result_2.set_index("device_id")

In [ ]:
print("# Final prediction")
scores = model.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result = pd.DataFrame(scores , columns=lable_group.classes_)
result["device_id"] = device_id
print(result.head(1))
result = result.set_index("device_id")

#result.to_csv('keras_' + str(log_loss(y_val, scores_val)) + '.csv', index=True, index_label='device_id')

In [ ]:
print("# Final prediction")
scores_4 = model_4.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_4 = pd.DataFrame(scores_4 , columns=lable_group.classes_)
result_4["device_id"] = device_id
print(result_4.head(1))
result_4 = result_4.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

In [ ]:
print("# Final prediction")
scores_5 = model_5.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_5 = pd.DataFrame(scores_5 , columns=lable_group.classes_)
result_5["device_id"] = device_id
print(result_5.head(1))
result_5 = result_5.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

In [ ]:
print("# Final prediction")
scores_6 = model_6.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_6 = pd.DataFrame(scores_6 , columns=lable_group.classes_)
result_6["device_id"] = device_id
print(result_6.head(1))
result_6 = result_6.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

In [ ]:
print("# Final prediction")
scores_7 = model_7.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_7 = pd.DataFrame(scores_7 , columns=lable_group.classes_)
result_7["device_id"] = device_id
print(result_7.head(1))
result_7 = result_7.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

In [ ]:
print("# Final prediction")
scores_8 = model_8.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_8 = pd.DataFrame(scores_8 , columns=lable_group.classes_)
result_8["device_id"] = device_id
print(result_8.head(1))
result_8 = result_8.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

In [ ]:
#print("# Final prediction")
#scores_9 = model_9.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
#result_9 = pd.DataFrame(scores_9 , columns=lable_group.classes_)
#result_9["device_id"] = device_id
#print(result_9.head(1))
#result_9 = result_9.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

In [ ]:
#tot = result + result_2 + result_3 + result_4 + result_5 + result_6
#tot = result_3 + result_5 + result_6 
#tot = result + result_2 + result_3 + result_4 + result_5 + result_6 + result_7
#tot = result + result_2 + result_3 + result_4 + result_5 + result_6 + result_7 + result_8 + result_9
tot = result + result_2 + result_3 + result_5 + result_6 + result_7 + result_8 + result_0 + result_01 + result_02

In [ ]:
#tot = tot/6
#tot = tot/3
#tot = tot/7
#tot = tot/8
tot = tot/10

In [ ]:
tot[tot<0.001]=0.001 # 1,2,3,4,5,5 models + this -> 2.23982
tot[tot>0.999]=0.999 # 1,2,3,4,5,5 models + this -> 2.23982

In [ ]:
#tot.to_csv('keras_' + str(log_loss(y_val, (scores_val + scores_val_2 + scores_val_3 + scores_val_4 + scores_val_5 + scores_val_6)/6)) + '.csv', index=True, index_label='device_id')

In [ ]:
tot.to_csv('keras_' + str(log_loss(y_val, (scores_val + scores_val_2 + scores_val_3 + scores_val_5 + scores_val_6 + scores_val_7 + scores_val_8)/1)) + '.csv', index=True, index_label='device_id')

In [ ]:
tot.to_csv('keras_' + str(log_loss(y_val, (scores_val + scores_val_2 + scores_val_3 + scores_val_4 + scores_val_5 + scores_val_6 + scores_val_7 + scores_val_8 + scores_val_9)/9)) + '.csv', index=True, index_label='device_id')